# Plotting DE results
-  PCA plots of results
-  MA plots of results
- Heatmaps of vsd normalized data

In [ ]:
suppressPackageStartupMessages(library('ComplexHeatmap'))
library('ggpubr')
library('hexbin')
library('tidyverse')
library('gridExtra')
library('viridis')
library('matrixStats')
library('corrr')
library('ggcorrplot')
library('DESeq2')
library('factoextra')
library('circlize')
library('RCurl')
library('ggtern')
library('ggfortify')
library('magick')
library('PCAtools')
library('multipanelfigure')
library('magrittr')
library('cowplot')

## 1. Read in data
read in vsd normalized data and log fold change data for all data and kegg-summerized counts each. ko-definition table, ko-orf table and sub categories-definition table. 

In [ ]:
read.vsd.k <- function(org){
    vsd.k <- read.csv(paste('./vsd_files/', org,'vsd.k.csv',sep=''))
    colnames(vsd.k)[1] <- 'ko_id'
    colnames(vsd.k) <- gsub('X', '', colnames(vsd.k))
    vsd.k
    }

## vsd normalized data
vsd.4 = read.csv('./vsd_files/04vsd.csv')
vsd.4.k = read.vsd.k('04')

vsd.8 = read.csv('./vsd_files/08vsd.csv')
vsd.8.k = read.vsd.k('08')

vsd.6 = read.csv('./vsd_files/06vsd.csv')
vsd.6.k = read.vsd.k('06')

vsd.13 = read.csv('./vsd_files/13vsd.csv')
vsd.13.k = read.vsd.k('13')


#lfcShrink data
## 04:
lfc4.AvL <- read.csv("./de_res_files/lfc4.AvL.csv")
lfc4.k.AvL <- read.csv("./de_res_files/lfc4.k.AvL.csv")

## 08:
lfc8.AvL <- read.csv("./de_res_files/lfc8.AvL.csv")
lfc8.HvL <- read.csv("./de_res_files/lfc8.HvL.csv")
lfc8.AvH <- read.csv("./de_res_files/lfc8.AvH.csv")
lfc8.k.AvL <- read.csv("./de_res_files/lfc8.k.AvL.csv")
lfc8.k.HvL <- read.csv("./de_res_files/lfc8.k.HvL.csv")
lfc8.k.AvH <- read.csv("./de_res_files/lfc8.k.AvH.csv")
AvL.k8 = read.csv('./de_res_files/AvL8.k.csv')
## 06:
lfc6.AvL <- read.csv("./de_res_files/lfc6.AvL.csv")
lfc6.HvL <- read.csv("./de_res_files/lfc6.HvL.csv")
lfc6.AvH <- read.csv("./de_res_files/lfc6.AvH.csv")
lfc6.k.AvL <- read.csv("./de_res_files/lfc6.k.AvL.csv")
lfc6.k.HvL <- read.csv("./de_res_files/lfc6.k.HvL.csv")
lfc6.k.AvH <- read.csv("./de_res_files/lfc6.k.AvH.csv")

## 13:
lfc13.AvL <- read.csv("./de_res_files/lfc13.AvL.csv")
lfc13.HvL <- read.csv("./de_res_files/lfc13.HvL.csv")
lfc13.AvH <- read.csv("./de_res_files/lfc13.AvH.csv")
lfc13.k.AvL <- read.csv("./de_res_files/lfc13.k.AvL.csv")
lfc13.k.HvL <- read.csv("./de_res_files/lfc13.k.HvL.csv")
lfc13.k.AvH <- read.csv("./de_res_files/lfc13.k.AvH.csv")

## Ko definitions and mappings
ko.def = read.csv('../kegg_names/ko_def.csv')
ko.def$ko_id = str_replace(ko.def$ko_id, 'isip1a', 'isip_1a')
ko4 = read.csv('../kegg_names/ko4_ls.csv')
ko8 = read.csv('../kegg_names/ko8_ls.csv')
ko6 = read.csv('../kegg_names/ko6_ls.csv')
ko13 = read.csv('../kegg_names/ko13_ls.csv')
mod = read.csv('../kegg_names/all.mod.csv')
path = read.csv('../kegg_names/all.path.csv')

In [ ]:
#reorder vsd df's so they are correct in heatmaps
colnames(vsd.4.k)
vsd.4.k = vsd.4.k %>% select('ko_id','pFe21_9A','pFe21_9B','pFe21_9C','add_backA','add_backB')
colnames(vsd.4.k)
colnames(vsd.8.k)
vsd.8.k = vsd.8.k %>% select('ko_id','pFe19A','pFe19B','pFe19C','pFe21_9A','pFe21_9B','pFe21_9C','add_backB','add_backC')
colnames(vsd.8.k)
colnames(vsd.6.k)
vsd.6.k = vsd.6.k %>% select('ko_id','pFe19A','pFe19B','pFe19C','pFe21_9A','pFe21_9B','pFe21_9C','add_backA','add_backB','add_backC')
colnames(vsd.6.k)
colnames(vsd.13.k)
vsd.13.k = vsd.13.k %>% select('ko_id','pFe19A','pFe19B','pFe19C','pFe21_9A','pFe21_9B','pFe21_9C','add_backA','add_backB','add_backC')
colnames(vsd.13.k)

# MA plots
1. Normal ma
2. ma with summs to kegg module level
3. plotly ma
4. ?
## All Orfs MA

In [ ]:
nrow(filter(AvL.k8, padj<=0.05))
nrow(filter(AvL.k8, padj>=0.05))
nrow(filter(lfc8.k.AvL, padj<=0.05))
nrow(filter(lfc8.k.AvL, padj>=0.05))


In [ ]:
## Function to plot and lfcShrink de dataframe 
# color genes by sub_category

ma.plot <- function(lfc, title){
    p <- ggmaplot(
        lfc, 
        main = title,
        fdr = 0.05, fc = 2, 
        size = 0.4,
        palette = c("#B31B21", "#1465AC", "darkgray"),
        #genenames = as.vector(lfc$symbol),
        
        legend = "top", 
        font.legend = "bold",
        font.main = "bold",
        ggtheme = ggplot2::theme_minimal()) 
    ggsave(filename = paste(title, ".png", sep=""), p, path="/work/nclab/lucy/SAB/Annotation/figs", width = 10, height = 10)
    print(p)
}


In [ ]:
ma.plot(lfc4.AvL, "Outer Shelf Cylindrotheca Low Iron vs Iron Add Back")

In [ ]:
ma.plot(lfc8.AvL,  "Inner Shelf Cylindrotheca Low Iron vs Iron Add Back")
ma.plot(lfc8.AvH,   "Inner Shelf Cylindrotheca High Iron vs Iron Add Back")
ma.plot(lfc8.HvL,   "Inner Shelf Cylindrotheca High Iron vs Iron Low")

In [ ]:
ma.plot(lfc6.AvL,  "G. oceanica Low Iron vs Iron Add Back")
ma.plot(lfc6.AvH,  "G. oceanica High Iron vs Iron Add Back")
ma.plot(lfc6.HvL,  "G. oceanica High Iron vs Low Iron ")

In [ ]:
ma.plot(lfc13.AvL,  "E. huxleyi Low Iron vs Iron Add Back") # top= add back, bottom= low fe
ma.plot(lfc13.AvH,  "E. huxleyi High Iron vs Iron Add Back") # top= add back, bottom=low fe
ma.plot(lfc13.HvL,  "E. huxleyi High Iron vs Low Iron") #top=high fe, bottom = low fe

## Kegg-summerized counts MA
### labeling top # kegg genes (top DE) 

In [ ]:
#for kegg level summarized:
ma.kegg <- function(lfc, comparison, organism, name) {
    lfc <- left_join(lfc, ko.def, by='ko_id')
    p <- ggmaplot(
        lfc, 
        main = paste(comparison),
        font.main=c(15,'bold','black'),
        submain=organism,
        font.submain= c(15,'bold.italic','black'),
        fdr = 0.05, fc = 2, 
        size = 4,
        #top=30,
        palette = c("#B31B21", "#1465AC", "darkgray"),
        genenames = as.vector(lfc$symbol),
        select.top.method='fc',
        legend = "top",
        font.label = c("bold", 10), 
        label.rectangle = T,
        font.legend = "bold", ggtheme=theme_minimal())
            
    ggsave(filename = paste(name, ".png", sep=""),p,path="/work/nclab/lucy/SAB/Annotation/figs", width = 10, height = 10)
    p

}

In [ ]:
ma.kegg(lfc4.k.AvL, "Iron amendment vs Low iron",'C. closterium 4', "MA.4k")

In [ ]:
ma.kegg(lfc8.k.AvL,'Iron amendment vs Low iron','C. closterium 8', 'MA.8k.AL')
ma.kegg(lfc8.k.AvH,'High iron vs Iron amendment','C. closterium 8','MA.8k.AH')
ma.kegg(lfc8.k.HvL,'High vs Low iron','C. closterium 8', 'MA.8k.HL')

In [ ]:
ma.kegg(lfc6.k.AvL, 'Iron amendment vs Low iron', 'G. oceanica','MA.6k.AL')
ma.kegg(lfc6.k.AvH, "High Fe vs Fe Incubation", 'G. oceanica','MA.6k.AH')
ma.kegg(lfc6.k.HvL, "High Fe vs Low Fe", 'G. oceanica','MA.6k.HL')

In [ ]:
ma.kegg(lfc13.k.AvL, 'Low Fe vs Fe Incubation','Emiliania huxleyi', 'MA.13k.AL')
ma.kegg(lfc13.k.AvH, 'High Fe vs Fe Incubation', 'Emiliania huxleyi','MA.13k.AH') 
ma.kegg(lfc13.k.HvL, 'High Fe vs Low Fe', 'Emiliania huxleyi', 'MA.13k.HL')

### labeling genes of interest
I need to make a list of kegg genes I wish to point out. Maybe I can obtain this list from the heatmaps? Or I can select them based on processes I want to highlight. 
ideas for list so far:
- isip's
- nitrogen transporter
- ammonium transporter
- flavodoxin
- ferredoxin
- psi and psii

In [ ]:
genes.list=c(' fldA, nifF, isiB','isip_1','isip_2','isip_2a','isip_3',' petF', ' petJ',' nirA',' nirB',' NR', ' CA',
            ' psbA',' psbU',' psbQ',' psbO',' petC',' petH',' petJ',' hemL',' TC.FEV.OM',' psbM', ' GAPDH, gapA',
             ' FLRT',' E4.2.1.2B, fumC, FH',' psbD',' psaA',' psaB')

ma.kegg.manual <- function(lfc, comparison, organism, name) {
    lfc <- left_join(lfc, ko.def, by='ko_id')
    p <- ggmaplot(
        lfc, 
        main = paste(comparison),
        font.main=c(15,'bold','black'),
        submain=organism,
        font.submain= c(15,'bold.italic','black'),
        fdr = 0.05, #fc = 1, 
        size = 4,
        top=0,
        label.select=genes.list,
        #palette = c("#B31B21", "#1465AC", "darkgray"),
        genenames = as.vector(lfc$symbol),
        select.top.method='padj',
        legend = "top",
        font.label = c(15,"bold"), 
        font.x=15,
        font.y=15,
        label.rectangle = TRUE,
        font.legend = c(15,"bold"), ggtheme=theme_minimal())
            
   ggsave(filename = paste(name, ".png", sep=""), p, 
          path="/work/nclab/lucy/SAB/Annotation/figs", width = 10, height = 10)
    p

}

ma.kegg.manual(lfc4.k.AvL, 'Low Fe vs Fe Incubation','Cylindrotheca uga.04', 'MA.4.manual.AL')

ma.kegg.manual(lfc8.k.AvL, 'Low Fe vs Fe Incubation','Cylindrotheca uga.08', 'MA.8.manual.AL')
ma.kegg.manual(lfc8.k.AvH, 'High Fe vs Fe Incubation', 'Cylindrotheca uga.08','MA.8.manual.AH') 
ma.kegg.manual(lfc8.k.HvL, 'High Fe vs Low Fe', 'Cylindrotheca uga.08', 'MA.8.manual.HL')

ma.kegg.manual(lfc6.k.AvL, 'Low Fe vs Fe Incubation','Gephyrocapsa oceanica', 'MA.6.manual.AL')
ma.kegg.manual(lfc6.k.AvH, 'High Fe vs Fe Incubation', 'Gephyrocapsa oceanica','MA.6.manual.AH') 
ma.kegg.manual(lfc6.k.HvL, 'High Fe vs Low Fe', 'Gephyrocapsa oceanica', 'MA.6.manual.HL')

ma.kegg.manual(lfc13.k.AvL, 'Low Fe vs Fe Incubation','Emiliania huxleyi', 'MA.13.manual.AL')
ma.kegg.manual(lfc13.k.AvH, 'High Fe vs Fe Incubation', 'Emiliania huxleyi','MA.13.manual.AH') 
ma.kegg.manual(lfc13.k.HvL, 'High Fe vs Low Fe', 'Emiliania huxleyi', 'MA.13.manual.HL')

In [ ]:
#vsd.13=pivot_longer(vsd.13,cols=!orfs,names_to = 'treatment', values_to='counts')
vsd.13$treatment=str_remove(vsd.13$treatment, '[ABC]')

vsd.13$iron[vsd.13$treatment=='pFe19']='High Fe'
vsd.13$iron[vsd.13$treatment=='pFe21_9']='Low Fe'
vsd.13$iron[vsd.13$treatment=='add_back']='Add Back'
head(vsd.13)

# Heatmaps 

## all orfs

In [ ]:
heat.all=function(vsd, org, name){
    mat_var <- column_to_rownames(vsd,'ko_id' )%>% as.matrix #turn into matrix
    mat_var <- (mat_var - rowMeans(mat_var) ) / rowSds(mat_var) #z-score

    col.anno <- data.frame('id' = colnames(vsd)[-1])
    col.anno$Organism <- str_extract(col.anno$id, '\\d{1,2}')
    col.anno$Treatment <- str_extract(col.anno$id, '((pFe\\d{1,2}|add)_(9|back)|pFe19)')
    col.anno$rep <- str_extract(col.anno$id, '[ABC]')
    col.anno <- HeatmapAnnotation(
        Treatment=col.anno$Treatment, 
        col = list(Treatment=c('pFe19' = "black", 'pFe21_9' = "white", 'add_back' = "grey"))) 

    png(file = paste('../figs/heat.k.all.', org, '.png', sep=''),width = 600,height = 800)
    ht = Heatmap(mat_var,cluster_columns = T, 
                 cluster_rows=T, show_row_dend = F,
                 column_title = paste(name), #'all Kegg genes', sep='\n'),
           show_column_names = FALSE,
                 col=magma(100),
            show_row_names = F, 
            top_annotation = col.anno, 
            heatmap_legend_param=list(title="Z-scored \nVSD \nnormalized \ncounts",
                                        position='topcenter'))
    
    draw(ht)
    dev.off()
    ht
}
all.4 = heat.all(vsd.4.k, '04', 'C. closeterium UGA4')

In [ ]:
all.8 = heat.all(vsd.8.k, '08', 'C. closterium UGA8')

In [ ]:
all.6 = heat.all(vsd.6.k, '06', 'G. oceanica')

In [ ]:
all.13 = heat.all(vsd.13.k, '13', 'G. huxleyi')

In [ ]:
all_heat=all.8+all.4+all.6+all.13


draw(all_heat, 
    heatmap_legend_side='bottom',merge_legends=T)

## Kegg-summed orfs 

In [ ]:
set.seed(123)
pvalue = 10^-runif(10, min = 0, max = 3)
is_sig = pvalue < 0.01
pch = rep("*", 10)
pch[!is_sig] = ''
pch
is_sig

In [ ]:
heat.map <- function(vsd, org, name){
    vsd <- vsd[0:50,] #subset top 30 genes
    mat_var <- column_to_rownames(vsd,'ko_id' )%>% as.matrix #turn into matrix
    mat_var <- (mat_var - rowMeans(mat_var) ) / rowSds(mat_var) #z-score

    col.anno <- data.frame('id' = colnames(vsd)[-1])
    col.anno$Organism <- str_extract(col.anno$id, '\\d{1,2}')
    col.anno$Treatment <- str_extract(col.anno$id, '((pFe\\d{1,2}|add)_(9|back)|pFe19)')
    col.anno$rep <- str_extract(col.anno$id, '[ABC]')
    col.anno <- HeatmapAnnotation(Treatment=col.anno$Treatment, col = list(Treatment=c(
            'pFe19' = "black", 
            'pFe21_9' = "white", 
            'add_back' = "grey"))) 
    
    row_names <- left_join(vsd, ko.def, by='ko_id') %>%
                        select('ko_id', 'symbol', 'name')
   new_names <- structure(row_names$symbol,  names=row_names$ko_id) #rename rows with kegg symbols

    
   png(file = paste('../figs/heat.top.', org, '.png', sep=''),width = 800,height = 800)
    
   ht=Heatmap(mat_var,cluster_columns = FALSE,column_title = name,
              col=viridis(1000),
           show_column_names = FALSE,
            row_labels = new_names[rownames(mat_var)],
            top_annotation = col.anno, #right_annotation=row.anno,
            row_names_gp = gpar(fontsize = 10),
            clustering_method_rows = "complete",
            heatmap_legend_param=list(title="Z-scored \nVSD \nnormalized \ncounts",
                                        position='topcenter'))
    
    draw(ht)
    dev.off()
    ht
    
}
vsd4= select(vsd.4.k, !'pFe21_9C')
 heat.map(vsd.4.k, '04', 'C. closterium 4')


In [ ]:
heat.map(vsd.8.k, '08', 'C. closterium 8')

In [ ]:
heat.map(vsd.6.k, '06', 'G. oceanica')

In [ ]:

heat.map(vsd.13.k, '13', 'G. huxleyi')

## Protein-specific heatmaps
<b/>Using module and pathway df's created in kegg script.</b>
### 1. Create a table with significance from DE.


In [ ]:

get.sig=function(HIGH,ADD){
    hvl.sig=filter(HIGH, padj<0.05)
    hvl.sig=data.frame('ko_id'=hvl.sig$ko_id, 'HvL'='*')
    avl.sig=filter(ADD, padj<0.05)
    avl.sig=data.frame('ko_id'=avl.sig$ko_id, 'AvL'='*')
    sig.ko=full_join(hvl.sig, avl.sig)
    ko.missing=ko.def[(ko.def$ko_id%in%sig.ko$ko_id==F), ]
    ko.missing=data.frame(ko_id=ko.missing$ko_id, 'HvL'=' ','AvL'=' ')
    sig.final=bind_rows(sig.ko, ko.missing)
    #sig.final=filter(sig.final,(ko_id %in% ko.def$ko_id))
    sig.final[is.na(sig.final)]=' '
    sig.final
    }
sig.ko13=get.sig(lfc13.k.HvL,lfc13.k.AvL)
sig.ko8=get.sig(lfc8.k.HvL, lfc8.k.AvL)
sig.ko6=get.sig(lfc6.k.HvL, lfc6.k.AvL)
sig.ko4={
    avl.sig=filter(lfc4.k.AvL, padj<0.05)
    sig.ko=data.frame('ko_id'=avl.sig$ko_id,'HvL'=' ', 'AvL'='*')
    ko.missing=ko.def[(ko.def$ko_id%in%sig.ko$ko_id==F), ]
    ko.missing=data.frame(ko_id=ko.missing$ko_id, 'HvL'=' ','AvL'=' ')
    sig.final=bind_rows(sig.ko, ko.missing)
    #sig.final=filter(sig.final,(ko_id %in% ko.def$ko_id))
    sig.final[is.na(sig.final)]=' '
    sig.final}


### 2. Make preliminary annotations

In [ ]:
####      call these functions in figure making functions to make the row and column annotations ####

col.anno = function(vsd, org){
    col.anno <- data.frame('id' = colnames(vsd)[-1])
    col.anno <- data.frame('id'=col.anno$id,
                           'Treatment'=str_extract(col.anno$id,'((pFe\\d{1,2}|add)_(9|back)|pFe19)'))
    if (org=='Gephyrocapsa \nhuxleyi'){
        col.org <- "#3456CA"
        isolate='13'
    }else if (org=='Gephyrocapsa \noceanica'){
        col.org <- "#90A1DA"
        isolate='06'
    }else if (org=='Cylindrotheca \nclosterium UGA4'){
        col.org <- "#0F6104"
        isolate='04'
    }else{
        col.org <- "#9AB18D"
        isolate='08'}
    if (org == "Gephyrocapsa \nhuxleyi"){
                legend.show=TRUE
            }else{legend.show=FALSE}
    
    col.anno <- HeatmapAnnotation(
        'Isolate'=anno_block(
            gp=gpar(fill=col.org, col=col.org),
            labels=org,
            labels_gp=gpar(fontface='bold.italic', col='black', fontsize=25)),
        'Treatment'=anno_simple(col.anno$Treatment, 
            col = c(
                 'pFe19' = "black",
                 'pFe21_9' = "white",
                  'add_back' = "grey")),
        annotation_name_gp=gpar(fontsize=25),
        show_annotation_name=legend.show) }



### 3. Set up Pathway data 
#### I. Read in Module and Pathway information.
This will subset genes (KO's) by the pathway or module associated with them.


#### II. Filter list for genes expressed by isolates
Not all ko's in each path will be expressed, filter out those so that pathway lists contain ko's expressed by at least one organism

In [ ]:
all.path = read.csv('../kegg_names/pathwaysHeatMap.csv')
a=unique(vsd.4.k$ko_id)
a=a[a%in%all.path$ko_id]

b=unique(vsd.8.k$ko_id)
b=b[b%in%all.path$ko_id]

c=unique(vsd.6.k$ko_id)
c=c[c%in%all.path$ko_id]

d=unique(vsd.13.k$ko_id)
d=d[d%in%all.path$ko_id]


length(b)
length(c)
length(d)
length(a)
x=union(a,b)
y=union(c,d)
z=union(x,y)

length(x)
length(y)
length(z)
all.path=all.path[(all.path$ko_id)%in%z,]

dim(all.path)
filter(ko.def, ko_id=='K08717')
filter(ko.def, ko_id=='K08716')


#### III. Adjust subcategory names and gene names for plotting later

In [ ]:
addGenes=filter(ko.def, ko_id %in%c('K03320','K01427','K04564','K04565','K08717','K08716'))
addGenes$name=str_remove(addGenes$name, '\\[.*\\]')
addGenes= mutate(addGenes, Path='map1111', 
                 sub_category=c('Nitrogen transporters', 'Nitrogen transporters','Superoxide dismutase','Nitrogen transporters','Superoxide dismutase','Urea cycle'),
                 broad_category=c('Nitrogen metabolism','Nitrogen metabolism','Photosynthesis','Nitrogen metabolism','Photosynthesis','Nitrogen metabolism'),
                 path_name=c('Nitrogen metabolism','Nitrogen metabolism','Photosynthesis','Nitrogen metabolism','Photosynthesis','Nitrogen metabolism'))
isip=ko.def[str_detect(ko.def$symbol,'isip.*')==T,]
isip=mutate(isip,Path='map1111', sub_category='ISIP', broad_category='Photosynthesis', path_name='Photosynthesis')
all.path=bind_rows(all.path, addGenes, isip)

all.path$name.2=str_squish(all.path$name)
tail(all.path,10)

In [ ]:
#filter(all.path, sub_category=='Urea cycle')
all.path[all.path$symbol==' ureG','name.2']='urease G'
all.path[all.path$symbol==' ureF','name.2']='urease F'
all.path[all.path$symbol=='CPS1','name.2']='carbamoyl-phosphate synthase 1'
all.path[all.path$symbol=='SLC14A','name.2']='SCF 14 urea transporter'
#filter(all.path, sub_category=='Superoxide dismutase')
all.path$name.2 = str_replace(all.path$name.2,'.*superoxide dismutase,','SOD')

#filter(all.path, sub_category=='Electron transport chain')
all.path[all.path$symbol=='petC','name.2'] = 'cytochrome b6f iron-sulfur subunit'

#filter(all.path, sub_category=='PSI')
all.path$name.2 = str_replace(all.path$name.2, 'psa', 'PSI')

filter(all.path, sub_category=='PSII')
all.path$name.2 = str_replace(all.path$name.2, 'psb', 'PSII')
all.path$name.2 = str_remove(all.path$name.2, 'reaction center')
all.path$name.2 = str_replace(all.path$name.2, 'oxygen-evolving enhancer', 'OEC')
all.path$name.2 = str_replace(all.path$name.2, '13kDa', 'Psb28')
#filter(all.path, sub_category%in%c('LHCA','LHCB'))
all.path$name.2 = str_remove(all.path$name.2, 'Chl a/b binding protein ')
all.path[str_detect(all.path$sub_category, 'LHCA'), 'sub_category']='PSI'
all.path[str_detect(all.path$sub_category, 'LHCB'), 'sub_category']='PSII'
#filter(all.path, sub_category=='Xanthophyll cycle')

#filter(all.path, sub_category=='Nitrite reductase')
all.path$sub_category = str_replace(all.path$sub_category, 'Nitrite', 'Nitrate/Nitrite')

filter(all.path, sub_category=='Nitrate/Nitrite reductase')
all.path$name.2 = str_remove(all.path$name.2, 'catalytic subunit')

#filter(all.path, sub_category=='Nitrogen transporters')
all.path$name.2 = str_remove(all.path$name.2, '.system')
all.path$name.2= str_replace(all.path$name.2, 'nitrate/nitrite transport substrate-binding protein', 'nitrate/nitrite transport substrate-binding\nprotein')
filter(all.path, sub_category=='Nitrogen transporters')

filter(all.path, sub_category=='GS/GOGAT')
all.path[str_detect(all.path$name.2,'glutamate dehydrogenase'),'sub_category'] = 'GDH'
all.path$sub_category = str_remove(all.path$sub_category, ' and GDH')
all.path[str_detect(all.path$symbol,'ncd2|CYP55'),'sub_category'] = 'Nitrogen recycling'
filter(all.path, sub_category=='GDH')
all.path$name.2 = str_replace(all.path$name.2, 'dehydrogenase \\(oxaloacetate', 'dehydrogenase\n\\(oxaloacetate')

In [ ]:
filter(ko.def, ko_id %in% c('K01601','K01602','K19269','K11517','K03781','K14272','K00600','K00830','K15893,K15919','K15918',
                            'K00281','K00605','K00382','K02437'))

In [ ]:
col_fun=colorRamp2(c(-2,0,2), hcl_palette ='Inferno')                             
ld1=Legend(col_fun = col_fun, title="Z-scored VST\nnormalized\ncounts", 
           labels_gp=gpar(fontsize=30), title_gp=gpar(fontsize=35,fontface='bold'), direction='horizontal')
ld2=Legend(labels=c('High iron','Low iron','Iron amendment'),
           legend_gp=gpar(fill=c('High iron'='black','Low iron'='white','Iron amendment'='grey')),
           labels_gp=gpar(fontsize=30),grid_width=unit(0.7,'cm'),
           title_gp=gpar(fontsize=35, fontface='bold'),
           title='Treatment', direction='horizontal')
ld3=Legend(labels="< 0.05", title='Significance', type='points', pch=8,
           labels_gp=gpar(fontsize=30),
           title_gp=gpar(fontsize=35,fontface='bold'))
                           
sub.path.heat = function(mp, vsd, org, sig.ko){
    #filter our ko's for pathway of interest
    select.cat=filter(all.path, path_name==mp)
    select.cat=select.cat %>% group_by(sub_category)
    select.cat = select.cat %>% arrange(sub_category, ko_id)
    #filter genes expressed in path
    genes.expressed <- vsd %>% filter(ko_id %in% select.cat$ko_id)
    # path genes not found in sample 
    genes.missing <- select.cat %>% filter(ko_id %in% vsd$ko_id==F) %>% select('ko_id')
    #  dataframe including genes not found replaced with an NA
    vsd.cat <- bind_rows(genes.expressed,genes.missing) 
    
    #create ordered vector of ko symbols to use as row names
    new_names= structure(select.cat$name.2, names=select.cat$ko_id)
 
    vsd.cat = vsd.cat[match(select.cat$ko_id, vsd.cat$ko_id),-which(names(vsd.cat)=='sub_category')]
    rownames(vsd.cat)=NULL
    
    mat_var <- column_to_rownames(vsd.cat,'ko_id' )%>% as.matrix 
    mat_var <- (mat_var - rowMeans(mat_var) ) / rowSds(mat_var)  #mat_var <- mat_var[sort(rownames(mat_var)),]
    #call function to make column annotation
    col.anno <- col.anno(vsd = vsd, org=org) 
    #call function to make row annotation
    sig.expressed=filter(sig.ko, ko_id%in%vsd.cat$ko_id)
    sig.expressed = sig.expressed[match(vsd.cat$ko_id, sig.expressed$ko_id),]
    row.anno <- rowAnnotation(
        ' Add'=anno_text(sig.expressed$AvL, gp = gpar(fontsize=30), show_name=TRUE), 
        ' High'=anno_text(sig.expressed$HvL, gp = gpar(fontsize=30), show_name=TRUE),
    annotation_name_gp=gpar(fontsize=20))
    
    if (org == "Gephyrocapsa \nhuxleyi"){
                legend.show=TRUE
            }else{legend.show=FALSE}
   
    ht=Heatmap(mat_var,show_heatmap_legend = F, col=inferno(100), 
               row_labels = new_names[rownames(mat_var)], row_names_gp = gpar(fontsize = 20), cluster_rows=FALSE,
               cluster_columns=FALSE, show_column_names=FALSE,
               top_annotation = col.anno, 
               right_annotation=row.anno,
               na_col = 'lightgrey')
   ht
    
   }

sub.cat.anno = function(path,col_list){
    sub_cat_leg= filter(all.path, path_name==path) %>% group_by(sub_category)
    sub_cat_leg = sub_cat_leg %>% arrange(sub_category, ko_id)
    sub_cat_mat=sub_cat_leg$sub_category
    print(unique(sub_cat_leg$sub_category))
    names(sub_cat_mat)=sub_cat_leg$name.2
    
    ht=Heatmap(sub_cat_mat, name='Subcategory', col=col_list,
           show_column_names=F, show_heatmap_legend=F, row_names_gp = gpar(fontsize = 20))
    ht
    }

In [ ]:
col_fun=colorRamp2(c(-2,0,2), hcl_palette ='Inferno')          

ld1=Legend(col_fun = col_fun, title="Z-scored VSD\nnormalized\ncounts", 
           title_gp=gpar(fontface='bold', fontsize=25),
           title_position='topcenter', direction='horizontal')

ld2=Legend(labels=c('High iron','Low iron','Iron amendment'),
           legend_gp=gpar(fill=c('High iron'='black','Low iron'='white','Iron amendment'='grey')),
            title_gp=gpar(fontface='bold'),
           grid_width=unit(0.7,'cm'),title='Treatment', direction='horizontal')

ld3=Legend(labels="< 0.05", title='Significance', type='points', pch=8,
            title_gp=gpar(fontface='bold'))
#photoresp= c('K01596','K01610','K01689','K01834','K15633','K15634','K15635','K00927','K00134','K00150',
 #           'K01803','K01623','K01624','K11645','K03841','K02446','K11532','K01086','K04041','K01622')

photoresp=c('K00844','K12407','K00845','K25026','K00886','K08074','K00918','K01810','K06859','K13810','K15916',
            'K00850','K16370','K21071','K00918','K01623','K01624','K11645','K16305','K16306','K01803',
            'K00134','K00150','K00927','K11389','K01834','K15633','K15634','K15635','K01689','K00873','K12406')
all.path.heat = function(vsd, org, sig.ko){
    #filter out ko's for pathway of interest
    select.cat=filter(ko.def, ko_id%in%photoresp)
    select.cat = select.cat %>% arrange(ko_id)
     #filter genes expressed in path
    genes.expressed <- vsd %>% filter(ko_id %in% select.cat$ko_id)
    # path genes not found in sample 
    genes.missing <- select.cat %>% filter(ko_id %in% vsd$ko_id==F) %>% select('ko_id')
    #  dataframe including genes not found replaced with an NA
    vsd.cat <- bind_rows(genes.expressed,genes.missing) 
    #create ordered vector of ko symbols to use as row names
    new_names= structure(select.cat$name, names=select.cat$ko_id)
    
    vsd.cat = vsd.cat %>% arrange(ko_id)
    mat_var <- column_to_rownames(vsd.cat,'ko_id' )%>% as.matrix     
    mat_var <- (mat_var - rowMeans(mat_var) ) / rowSds(mat_var) 
    #call function to make column annotation
    col.anno <- col.anno(vsd = vsd, org=org) 
    #call function to make row annotation
    sig.expressed=filter(sig.ko, ko_id%in%vsd.cat$ko_id)
    sig.expressed = sig.expressed[match(vsd.cat$ko_id, sig.expressed$ko_id),]
    row.anno <- rowAnnotation(
        ' Add'=anno_text(sig.expressed$AvL,show_name=TRUE),  
        ' High'=anno_text(sig.expressed$HvL, show_name=TRUE))
    
    if (org == "Gephyrocapsa \nhuxleyi"){
                legend.show=TRUE
            }else{legend.show=FALSE}
   
    ht=Heatmap(mat_var,show_heatmap_legend = F, 
               col=inferno(100), 
               row_labels = new_names[rownames(mat_var)],
               cluster_row_slices=F, row_title_gp = gpar(fontface='bold'),
               cluster_columns = FALSE,
               cluster_rows=FALSE,
               show_column_names = FALSE,
               top_annotation = col.anno, 
               right_annotation=row.anno,
               na_col = 'lightgrey')
   ht
    
   }


all.4=all.path.heat(vsd=vsd.4.k,org = 'Cylindrotheca \nclosterium UGA4',sig.ko = sig.ko4)
all.8=all.path.heat(vsd=vsd.8.k,org = 'Cylindrotheca \nclosterium UGA8',sig.ko = sig.ko8)
all.6=all.path.heat(vsd=vsd.6.k,org = 'Gephyrocapsa \noceanica',sig.ko = sig.ko6)
all.13=all.path.heat(vsd=vsd.13.k,org = 'Gephyrocapsa \nhuxleyi',sig.ko = sig.ko13)


all.list=all.8+all.4+all.6+all.13
draw(all.list)
draw(all.8+all.4)
draw(all.13)

## make all pathways into one heatmap

In [ ]:
col_fun=colorRamp2(c(-2,0,2), hcl_palette ='Inferno')          

ld1=Legend(col_fun = col_fun, title="Z-scored VSD\nnormalized\ncounts", 
           title_gp=gpar(fontface='bold', fontsize=35), labels_gp=gpar(fontsize=30),
           title_position='topcenter', direction='horizontal')

ld2=Legend(labels=c('High iron','Low iron','Iron amendment'),
           legend_gp=gpar(fill=c('High iron'='black','Low iron'='white','Iron amendment'='grey')),
           labels_gp=gpar(fontsize=30), title_gp=gpar(fontface='bold', fontsize=35),
           grid_width=unit(0.7,'cm'),title='Treatment', direction='horizontal')

ld3=Legend(labels="< 0.05", title='Significance', type='points', pch=8,
           labels_gp=gpar(fontsize=30), title_gp=gpar(fontface='bold', fontsize=35))

all.path.heat = function(subs, vsd, org, sig.ko){
    #filter out ko's for pathway of interest
    select.cat=filter(all.path, sub_category%in%subs)
    select.cat=select.cat %>% group_by(path_name, sub_category)
    select.cat = select.cat %>% arrange(desc(path_name), sub_category, ko_id)
    #filter genes expressed in path
    genes.expressed <- vsd %>% filter(ko_id %in% select.cat$ko_id)
    # path genes not found in sample 
    genes.missing <- select.cat %>% filter(ko_id %in% vsd$ko_id==F) %>% select('ko_id')
    #  dataframe including genes not found replaced with an NA
    vsd.cat <- bind_rows(genes.expressed,genes.missing) 
    
    #create ordered vector of ko symbols to use as row names
    new_names= structure(select.cat$name.2, names=select.cat$ko_id)
 
    vsd.cat = vsd.cat[match(select.cat$ko_id, vsd.cat$ko_id),-which(names(vsd.cat)%in%c('path_name','sub_category'))]
    rownames(vsd.cat)=NULL

    mat_var <- column_to_rownames(vsd.cat,'ko_id' )%>% as.matrix 
    mat_var <- (mat_var - rowMeans(mat_var) ) / rowSds(mat_var)  
    #call function to make column annotation
    col.anno <- col.anno(vsd = vsd, org=org) 
    #call function to make row annotation
    sig.expressed=filter(sig.ko, ko_id%in%vsd.cat$ko_id)
    sig.expressed = sig.expressed[match(vsd.cat$ko_id, sig.expressed$ko_id),]
    row.anno <- rowAnnotation(
        ' Add'=anno_text(sig.expressed$AvL,show_name=TRUE, gp = gpar(fontsize=50)),  
        ' High'=anno_text(sig.expressed$HvL, show_name=TRUE, gp = gpar(fontsize=50)), 
    annotation_name_gp=gpar(fontsize=30))
    
    if (org == "Gephyrocapsa \nhuxleyi"){
                legend.show=TRUE
            }else{legend.show=FALSE}
   
    ht=Heatmap(mat_var,show_heatmap_legend = F, 
               col=inferno(100), #row_split=factor(select.cat$path_name,levels=c('Photosynthesis', 'Nitrogen metabolism')),
               row_labels = new_names[rownames(mat_var)], 
               #row_gap=unit(2, 'cm'), cluster_row_slices=F, row_title_gp = gpar(fontsize=50,fontface='bold'),
               cluster_columns = FALSE, cluster_rows=FALSE, show_column_names = FALSE,
               top_annotation = col.anno, right_annotation=row.anno, na_col = 'lightgrey')
   ht
    
   }

sub.cat.anno = function(subs,col_list){
    sub_cat_leg= filter(all.path, sub_category%in%subs) %>% group_by(path_name,sub_category)
    sub_cat_leg = sub_cat_leg %>% arrange(desc(path_name), sub_category, ko_id)
    sub_cat_mat=sub_cat_leg$sub_category
    print(unique(sub_cat_leg$sub_category))
    names(sub_cat_mat)=sub_cat_leg$name.2
    
    ht=Heatmap(sub_cat_mat, name='Subcategory', col=col_list, 
               #row_split=factor(sub_cat_leg$path_name, levels=c('Photosynthesis', 'Nitrogen metabolism')),
               #row_gap=unit(2,'cm'), cluster_row_slices=F,
               show_column_names=F, show_heatmap_legend=F,
                row_names_gp = gpar(fontsize = 30))
    ht
    }


In [ ]:
#unique(all.path$sub_category)
sub_cat=c('Electron transport chain','ISIP','PSI','PSII','Superoxide dismutase','Xanthophyll cycle',
          'GDH','GS/GOGAT','Nitrate/Nitrite reductase','Nitrogen recycling','Nitrogen transporters','Urea cycle')
sub_col=c('Electron transport chain'='#8D0707','ISIP'='#CC0025','PSI'='#FF0462','PSII'='#FF3CAA',
          'Superoxide dismutase'='#FF74DC','Xanthophyll cycle'='#FFACF9',
          'GDH'='#0C0057','GS/GOGAT'='#4C2BDE','Nitrate/Nitrite reductase'='#659BB8','Nitrogen recycling'='#00D1BC',
          'Nitrogen transporters'='#B7F39B','Urea cycle'='#FFF7C7'
           )
ld4 = Legend(labels=c('Electron transport chain','ISIP','PSI','PSII','Superoxide dismutase','Xanthophyll cycle',
          'GDH','GS/GOGAT','Nitrate/Nitrite reductase','Nitrogen recycling','Nitrogen transporters','Urea cycle'),
             legend_gp=gpar(fill=sub_col), grid_width=unit(0.7,'cm'),
             labels_gp=gpar(fontsize=25),
             title_gp=gpar( fontface='bold',fontsize=25),
             title='Module', ncol=3,
            title_position='topcenter', gap=unit(2,'cm'))


all.4=all.path.heat(subs=sub_cat,vsd=vsd.4.k,org = 'Cylindrotheca \nclosterium UGA4',sig.ko = sig.ko4)
all.8=all.path.heat(subs=sub_cat,vsd=vsd.8.k,org = 'Cylindrotheca \nclosterium UGA8',sig.ko = sig.ko8)
all.6=all.path.heat(subs=sub_cat,vsd=vsd.6.k,org = 'Gephyrocapsa \noceanica',sig.ko = sig.ko6)
all.13=all.path.heat(subs=sub_cat,vsd=vsd.13.k,org = 'Gephyrocapsa \nhuxleyi',sig.ko = sig.ko13)

all.cat=sub.cat.anno(sub_cat, sub_col)
all.list=all.8+all.4+all.6+all.13+all.cat

png('../figs/combined_heatmap.png', width=2500, height=2300)
draw(all.list, annotation_legend_list=packLegend(ld1,ld2,ld3,ld4, direction='horizontal', column_gap = unit(1.5, 'cm')),
    heatmap_legend_side='bottom', merge_legends=T, padding=unit(c(10,2,2,150), 'mm'), adjust_annotation_extension=TRUE)
dev.off()


In [ ]:
#unique(all.path$sub_category)
sub_cat=c('Electron transport chain','ISIP','PSI','PSII','Superoxide dismutase','Xanthophyll cycle',
          'GDH','GS/GOGAT','Nitrate/Nitrite reductase','Nitrogen recycling','Nitrogen transporters','Urea cycle')
sub_col=c('Electron transport chain'='#284026',
          'ISIP'='#1A8118',
          'PSI'='#A39F24',
          'PSII'='#FFD666',
          'Superoxide dismutase'='#FE8E3E',
          'Xanthophyll cycle'='#882216',
          'GDH'='#EB0014',
          'GS/GOGAT'='#FF7ADC',
          'Nitrate/Nitrite reductase'='#A90AFF',
          'Nitrogen recycling'='#8E7AFF',
          'Nitrogen transporters'='#330AFF',
          'Urea cycle'='#00046B'
           )
ld4 = Legend(labels=c('Electron transport chain','ISIP','PSI','PSII','Superoxide dismutase','Xanthophyll cycle',
          'GDH','GS/GOGAT','Nitrate/Nitrite reductase','Nitrogen recycling','Nitrogen transporters','Urea cycle'),
             legend_gp=gpar(fill=sub_col), grid_width=unit(0.7,'cm'),
             labels_gp=gpar(fontsize=25),
             title_gp=gpar( fontface='bold',fontsize=25),
             title='Module', ncol=3,
            title_position='topcenter', gap=unit(2,'cm'))


all.4=all.path.heat(subs=sub_cat,vsd=vsd.4.k,org = 'Cylindrotheca \nclosterium UGA4',sig.ko = sig.ko4)
all.8=all.path.heat(subs=sub_cat,vsd=vsd.8.k,org = 'Cylindrotheca \nclosterium UGA8',sig.ko = sig.ko8)
all.6=all.path.heat(subs=sub_cat,vsd=vsd.6.k,org = 'Gephyrocapsa \noceanica',sig.ko = sig.ko6)
all.13=all.path.heat(subs=sub_cat,vsd=vsd.13.k,org = 'Gephyrocapsa \nhuxleyi',sig.ko = sig.ko13)

all.cat=sub.cat.anno(sub_cat, sub_col)
all.list=all.8+all.4+all.6+all.13+all.cat

png('../figs/combined_heatmap.png', width=2500, height=2300)
draw(all.list, annotation_legend_list=packLegend(ld1,ld2,ld3,ld4, direction='horizontal', column_gap = unit(1.5, 'cm')),
    heatmap_legend_side='bottom', merge_legends=T, padding=unit(c(10,2,2,150), 'mm'), adjust_annotation_extension=TRUE)
dev.off()


In [ ]:
c_sub=c('CAM dark','CAM light','C4 Dicarboxilic acid cycle','Calvin cycle','Carbonic anhydrase')
c_col=c('CAM dark'='darkblue','CAM light'='lightblue','C4 Dicarboxilic acid cycle'='lightgreen',
        'Calvin cycle'='darkgreen','Carbonic anhydrase'='pink')

carbon.4=all.path.heat(subs=c_sub,vsd=vsd.4.k,org = 'Cylindrotheca \nclosterium UGA4',sig.ko = sig.ko4)
carbon.8=all.path.heat(subs=c_sub,vsd=vsd.8.k,org = 'Cylindrotheca \nclosterium UGA8',sig.ko = sig.ko8)
carbon.6=all.path.heat(subs=c_sub,vsd=vsd.6.k,org = 'Gephyrocapsa \noceanica',sig.ko = sig.ko6)
carbon.13=all.path.heat(subs=c_sub,vsd=vsd.13.k,org = 'Gephyrocapsa \nhuxleyi',sig.ko = sig.ko13)

carbon.cat=sub.cat.anno(c_sub, c_col)
carbon.list=carbon.8+carbon.4+carbon.6+carbon.13+carbon.cat

c.ld4 = Legend(labels=c('CAM dark','CAM light','C4 Dicarboxilic acid cycle','Calvin cycle','Carbonic anhydrase'),
             legend_gp=gpar(fill=c_col), grid_width=unit(0.7,'cm'),
             labels_gp=gpar(fontsize=25),
             title_gp=gpar( fontface='bold',fontsize=25),
             title='Module', ncol=3,
            title_position='topcenter', gap=unit(2,'cm'))

png('../figs/carbon_heatmap.png', width=2500, height=2300)
draw(carbon.list, annotation_legend_list=packLegend(ld1,ld2,ld3,c.ld4, direction='horizontal', column_gap = unit(1.5, 'cm')),
    heatmap_legend_side='bottom', merge_legends=T, padding=unit(c(10,2,2,150), 'mm'), adjust_annotation_extension=TRUE)
dev.off()


## Make smaller heatmaps for talk

In [ ]:
filter(all.path, path_name=='Photosynthesis')

In [ ]:
#select genes by ko_id or name then build heatmap
## for photosynthesis 10 genes pulled out
photo_talk=c('K08902','K08903','K02719','K08914','K08916','K09838','K09839','K04564','K04565')

talk.heat = function(ko, vsd, org, sig.ko){
    #filter out ko's for pathway of interest
    select.cat=filter(all.path, ko_id%in%ko)
    select.cat=select.cat %>% group_by(path_name, sub_category)
    select.cat = select.cat %>% arrange(desc(path_name), sub_category, ko_id)
    #filter genes expressed in path
    genes.expressed <- vsd %>% filter(ko_id %in% select.cat$ko_id)
    # path genes not found in sample 
    genes.missing <- select.cat %>% filter(ko_id %in% vsd$ko_id==F) %>% select('ko_id')
    #  dataframe including genes not found replaced with an NA
    vsd.cat <- bind_rows(genes.expressed,genes.missing) 
    
    #create ordered vector of ko symbols to use as row names
    new_names= structure(select.cat$name.2, names=select.cat$ko_id)
 
    vsd.cat = vsd.cat[match(select.cat$ko_id, vsd.cat$ko_id),-which(names(vsd.cat)%in%c('path_name','sub_category'))]
    rownames(vsd.cat)=NULL

    mat_var <- column_to_rownames(vsd.cat,'ko_id' )%>% as.matrix 
    mat_var <- (mat_var - rowMeans(mat_var) ) / rowSds(mat_var)  
    #call function to make column annotation
    col.anno <- col.anno(vsd = vsd, org=org) 
    #call function to make row annotation
    sig.expressed=filter(sig.ko, ko_id%in%vsd.cat$ko_id)
    sig.expressed = sig.expressed[match(vsd.cat$ko_id, sig.expressed$ko_id),]
    row.anno <- rowAnnotation(
        ' Add'=anno_text(sig.expressed$AvL,show_name=TRUE, gp = gpar(fontsize=50)),  
        ' High'=anno_text(sig.expressed$HvL, show_name=TRUE, gp = gpar(fontsize=50)), 
    annotation_name_gp=gpar(fontsize=30))
    
    if (org == "Gephyrocapsa \nhuxleyi"){
                legend.show=TRUE
            }else{legend.show=FALSE}
   
    ht=Heatmap(mat_var,show_heatmap_legend = F,  col=inferno(100),row_labels = new_names[rownames(mat_var)], 
               cluster_columns = FALSE, cluster_rows=FALSE, show_column_names = FALSE,
               top_annotation = col.anno, right_annotation=row.anno, na_col = 'lightgrey')
   ht
    
   }

sub.cat.anno = function(ko,col_list){
    sub_cat_leg= filter(all.path, ko_id%in%ko) %>% group_by(path_name,sub_category)
    sub_cat_leg = sub_cat_leg %>% arrange(desc(path_name), sub_category, ko_id)
    sub_cat_mat=sub_cat_leg$sub_category
    print(unique(sub_cat_leg$sub_category))
    names(sub_cat_mat)=sub_cat_leg$name.2
    
    ht=Heatmap(sub_cat_mat, name='Subcategory', col=col_list, 
                show_column_names=F, show_heatmap_legend=F,
                row_names_gp = gpar(fontsize = 30))
    ht
    }

photo_col=c('PSII'='#8A8EFF', 'Superoxide dismutase'='#060EE2','Xanthophyll cycle'='#050856')

p.talk.4 = talk.heat(photo_talk, vsd.4.k, 'Cylindrotheca \nclosterium UGA4', sig.ko4)
p.talk.8 = talk.heat(photo_talk, vsd.8.k, 'Cylindrotheca \nclosterium UGA8', sig.ko8)
p.talk.6 = talk.heat(photo_talk, vsd.6.k, 'Gephyrocapsa \noceanica', sig.ko6)
p.talk.13 = talk.heat(photo_talk, vsd.13.k, 'Gephyrocapsa \nhuxleyi', sig.ko13)

p.talk.cat = sub.cat.anno(photo_talk, photo_col)
p.talk.list = p.talk.8 + p.talk.4 + p.talk.6 + p.talk.13 + p.talk.cat

p.talk.ld4 = Legend(labels=c('PSII','Superoxide dismutase','Xanthophyll cycle'),
             legend_gp=gpar(fill=photo_col), grid_width=unit(0.7,'cm'), labels_gp=gpar(fontsize=30),
             title_gp=gpar( fontface='bold',fontsize=35), title='Module', title_position='topcenter')

png('../figs/short_photo_heatmap.png', width=2500, height=1000)
draw(p.talk.list, annotation_legend_list=packLegend(ld1,ld2,ld3,p.talk.ld4, direction='horizontal', column_gap = unit(1.5, 'cm')),
    heatmap_legend_side='bottom', merge_legends=T, padding=unit(c(2,2,2,80), 'mm'), adjust_annotation_extension=TRUE)
dev.off()

In [ ]:
sessionInfo()